In [1]:
#required imports

import numpy as np # library to handle data in a vectorized manner
import pandas as pd

import matplotlib.cm as cm
import matplotlib.colors as colors

import requests
from pandas.io.json import json_normalize

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

First, we load neighborhood data from https://en.wikipedia.org/wiki/Neighborhoods_of_Kansas_City,_Missouri

In [63]:
# The code was removed by Watson Studio for sharing.

,Neighborhood
0,18th and Vine
1,49-63 Coalition
2,Antioch Acres
3,Armour Fields
4,Armour Hills


In [62]:
kansas_city_data = df_data_1
kansas_city_data['Latitude'] = 0.0
kansas_city_data['Longitude'] = 0.0
kansas_city_data.head()


,Neighborhood,Latitude,Longitude
0,18th and Vine,0.0,0.0
1,49-63 Coalition,0.0,0.0
2,Antioch Acres,0.0,0.0
3,Armour Fields,0.0,0.0
4,Armour Hills,0.0,0.0


In [60]:
geolocator = Nominatim(user_agent="kc_explorer")
for index, row in kansas_city_data.iterrows():
    lat = 0.0
    long = 0.0
    neighborhood = row.Neighborhood
    address = neighborhood + ', Kansas City'
    try:
        location = geolocator.geocode(address)
    except: # getting some timeouts
        lat = 0.0
        long = 0.0

    if location is None:
        lat = 0.0
        long = 0.0
#        print('Bad address', address)
    else:
        lat = location.latitude
        long = location.longitude

    kansas_city_data.at[index,'Latitude'] = lat
    kansas_city_data.at[index,'Longitude'] = long
#print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))
#    row['Latitude'] = latitude
#    row['Longitude'] = longitude

In [58]:
kansas_city_data

,Neighborhood,Latitude,Longitude
0,18th and Vine,39.094440,-94.553886
1,49-63 Coalition,39.065361,-94.556665
2,Antioch Acres,38.994974,-94.683889
3,Armour Fields,39.004839,-94.598302
4,Armour Hills,39.004702,-94.588570
5,Ashland Ridge,39.057391,-94.473155
6,Bannister Acres,38.946507,-94.503142
13,Blue Hills,39.028717,-94.561469
14,Blue Hills Estates,38.895601,-94.603577
16,Blue Valley,39.100105,-94.578142


In [8]:
kansas_city_data = kansas_city_data[(kansas_city_data.Latitude != 0.0 ) & (kansas_city_data.Longitude != 0.0)]

In [9]:
kansas_city_data.shape

(171, 3)

Let's visualize the neighborhoods.  First we find the center of Kansas City:

In [10]:
address = 'Kansas City, Missouri'
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kansas City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kansas City are 39.100105, -94.5781416.


In [11]:
# create map of Kansas City using latitude and longitude values
map_kansas_city = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(kansas_city_data['Latitude'], kansas_city_data['Longitude'], kansas_city_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kansas_city)  
    
map_kansas_city

__Let's start exploring the Kansas City neighborhoods using Foursquare__

In [12]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
CLIENT_ID = 'FWXDOZKTXIXHYOUJ2EY2JJ4E0DOZAS01WEKTDFYOKEZR0DGV' # your Foursquare ID
CLIENT_SECRET = 'AOKQ3MDFNYE43R4Z0LJVKA4YS3J25IYOECNFGUAW43NHPY0U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FWXDOZKTXIXHYOUJ2EY2JJ4E0DOZAS01WEKTDFYOKEZR0DGV
CLIENT_SECRET:AOKQ3MDFNYE43R4Z0LJVKA4YS3J25IYOECNFGUAW43NHPY0U


In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
kansas_city_venues = getNearbyVenues(kansas_city_data.Neighborhood, kansas_city_data.Latitude, kansas_city_data.Longitude)
kansas_city_venues.head(10)

18th and Vine
49-63 Coalition
Antioch Acres
Armour Fields
Armour Hills
Ashland Ridge
Bannister Acres
Blue Hills
Blue Hills Estates
Blue Valley
Blue Vue Hills
Boone Hills
Boulevard Village
Bradford Place
Briarcliff
Briarcliff West
Bridlespur
Brookside
Calico Farms
CBD-Downtown
Center City
Chaumiere
Citadel
Claymont
Claymont North
Clayton
Coachlight Square
Coleman Highlands
Colonial Square
Columbus Park
Country Club
Country Club District
Country Club Plaza
Country Lane Estates
Countryside
Coves North
Crestwood
Crossgates
Crossroads
Davidson
Dunbar
Eastwood Hills
Fairlane
Gashland
Glen Lake
Glen Oaks
Greenwood
Hanover Place
Harlem
Hickman Mills
Hickman Mills South
Hidden Valley
Highland View
Hill Haven
Hillcrest
Holiday Hills
Holmes Park
Hospital Hill
Hyde Park
Independence Plaza
Indian Mound
Ingleside
Ivanhoe
Jefferson Highlands
KCI
Key Coalition
Kirkside
Knobtown
Knoches Park
Lakeview Terrace
Lea Manor
Leeds
Legacy East
Lewis Heights
Library District
Loma Vista
Longview
Lykins
Manheim P

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,18th and Vine,39.094440,-94.553886,Arthur Bryant's Barbeque,39.091483,-94.556056,BBQ Joint
1,18th and Vine,39.094440,-94.553886,Gates Bar-B-Q,39.098435,-94.555813,BBQ Joint
2,18th and Vine,39.094440,-94.553886,Family Dollar,39.090700,-94.551200,Discount Store
3,18th and Vine,39.094440,-94.553886,Shell,39.095086,-94.555961,Gas Station
4,18th and Vine,39.094440,-94.553886,Park Parade Liquor,39.095170,-94.555942,Liquor Store
5,18th and Vine,39.094440,-94.553886,Church's Chicken,39.098610,-94.551803,Fried Chicken Joint
6,18th and Vine,39.094440,-94.553886,The Pit Lounge,39.098569,-94.555851,Lounge
7,18th and Vine,39.094440,-94.553886,McDonald's,39.095438,-94.551091,Fast Food Restaurant
8,49-63 Coalition,39.065361,-94.556665,Walgreens,39.067700,-94.553855,Pharmacy
9,49-63 Coalition,39.065361,-94.556665,Pizza Hut,39.068701,-94.554296,Pizza Place


__Let's group the venues by neighborhood__

In [18]:
# one hot encoding
kansas_city_onehot = pd.get_dummies(kansas_city_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kansas_city_onehot['Neighborhood'] = kansas_city_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kansas_city_onehot.columns[-1]] + list(kansas_city_onehot.columns[:-1])
kansas_city_onehot = kansas_city_onehot[fixed_columns]

kansas_city_onehot.head()

,Neighborhood,ATM,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,...,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,18th and Vine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,18th and Vine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18th and Vine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,18th and Vine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,18th and Vine,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
kansas_city_onehot.shape

(1990, 242)

In [20]:
kansas_city_grouped = kansas_city_onehot.groupby('Neighborhood').mean().reset_index()
kansas_city_grouped

,Neighborhood,ATM,Accessories Store,Airport,Airport Terminal,American Restaurant,Antique Shop,Arcade,Art Gallery,Art Museum,...,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,18th and Vine,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
1,49-63 Coalition,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.111111,0.000000
2,Antioch Acres,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
3,Armour Fields,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.166667,0.0,0.00,0.000000,0.000000
4,Armour Hills,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
5,Ashland Ridge,0.055556,0.00000,0.0,0.0,0.055556,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
6,Blue Hills,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
7,Blue Hills Estates,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
8,Blue Valley,0.000000,0.00000,0.0,0.0,0.052632,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.026316,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000
9,Blue Vue Hills,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [21]:
num_top_venues = 5

for hood in kansas_city_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = kansas_city_grouped[kansas_city_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----18th and Vine----
                 venue  freq
0            BBQ Joint  0.25
1  Fried Chicken Joint  0.12
2         Liquor Store  0.12
3       Discount Store  0.12
4               Lounge  0.12


----49-63 Coalition----
                venue  freq
0                 Bar  0.11
1         Pizza Place  0.11
2  Chinese Restaurant  0.11
3      Clothing Store  0.11
4              Market  0.11


----Antioch Acres----
                venue  freq
0              Bakery   0.2
1      Cosmetics Shop   0.2
2  Mexican Restaurant   0.2
3     Doctor's Office   0.2
4    Asian Restaurant   0.2


----Armour Fields----
              venue  freq
0              Café  0.17
1          Wine Bar  0.17
2  Sculpture Garden  0.17
3              Park  0.17
4        Playground  0.17


----Armour Hills----
                           venue  freq
0                           Park  0.33
1                     Playground  0.33
2                          Trail  0.33
3  Paper / Office Supplies Store  0.00
4                  M

#### Let's put that into a *pandas* dataframe
First, let's write a function to sort the venues in descending order.

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kansas_city_grouped['Neighborhood']

for ind in np.arange(kansas_city_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kansas_city_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(200)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,18th and Vine,BBQ Joint,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Gas Station,Liquor Store,Lounge,Flea Market,Exhibit,Flower Shop
1,49-63 Coalition,Market,Pharmacy,Bar,Clothing Store,Grocery Store,Intersection,Chinese Restaurant,Pizza Place,Women's Store,Flea Market
2,Antioch Acres,Mexican Restaurant,Cosmetics Shop,Asian Restaurant,Doctor's Office,Bakery,Disc Golf,Farm,Food Service,Food Court,Food & Drink Shop
3,Armour Fields,Sculpture Garden,Playground,Wine Bar,Café,Theater,Park,Farmers Market,Event Space,Exhibit,Farm
4,Armour Hills,Playground,Park,Trail,Yoga Studio,Financial or Legal Service,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
5,Ashland Ridge,Sporting Goods Shop,Bar,ATM,Bank,Home Service,Grocery Store,Greek Restaurant,Fast Food Restaurant,Discount Store,Chinese Restaurant
6,Blue Hills,Park,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market,Fish & Chips Shop
7,Blue Hills Estates,Assisted Living,Hunan Restaurant,Mexican Restaurant,Diner,BBQ Joint,Furniture / Home Store,Golf Course,Yoga Studio,Farmers Market,Fast Food Restaurant
8,Blue Valley,Sandwich Place,Park,Nightclub,Coffee Shop,American Restaurant,BBQ Joint,Sports Bar,Bar,Museum,Beer Bar
9,Blue Vue Hills,Insurance Office,Discount Store,Yoga Studio,Event Space,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Flea Market


#### Let's isolate and map the locations of neighborhoods that have the sports bars now.

In [52]:
df = neighborhoods_venues_sorted
sb = 'Sports Bar'
f1 = (df['1st Most Common Venue'] == sb)
f2 = (df['2nd Most Common Venue'] == sb)
f3 = (df['3rd Most Common Venue'] == sb)
f4 = (df['4th Most Common Venue'] == sb)
f5 = (df['5th Most Common Venue'] == sb)
f5 = (df['5th Most Common Venue'] == sb)
f6 = (df['6th Most Common Venue'] == sb)
f7 = (df['7th Most Common Venue'] == sb)
f8 = (df['8th Most Common Venue'] == sb)
f9 = (df['9th Most Common Venue'] == sb)
f10 = (df['10th Most Common Venue'] == sb)
sports_bar_neighborhoods = df[ f1 | f2 | f3 | f4 | f5 | f6 | f7 | f8 | f9 | f10 ]['Neighborhood']
sports_bars_data = kansas_city_data[kansas_city_data['Neighborhood'].isin(sports_bar_neighborhoods)]
sports_bars_data


,Neighborhood,Latitude,Longitude
16,Blue Valley,39.100105,-94.578142
19,Boulevard Village,39.100105,-94.578142
72,Hanover Place,39.059623,-94.588059
79,Hill Haven,39.100105,-94.578142
80,Hillcrest,39.100105,-94.578142
115,Martin City,38.885271,-94.596325
133,Oakwood,39.100105,-94.578142
134,"Old Hyde Park Historic District, Inc.",39.100105,-94.578142
140,Park Forest,39.100105,-94.578142
142,Park Plaza,39.216051,-94.641819


### Create a map of Kansas City neighborhoods that currently have sports bars as top-10 venue

In [64]:
# create map of Kansas City using latitude and longitude values
map_kansas_city_sports_bars = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(sports_bars_data['Latitude'], sports_bars_data['Longitude'], sports_bars_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kansas_city_sports_bars)  
    
map_kansas_city_sports_bars

### Cluster Neighborhoods
Run *k*-means to cluster the neighborhood into 5 clusters.

In [36]:
# set number of clusters
kclusters = 5

kansas_city_grouped_clustering = kansas_city_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kansas_city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 4, 1, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [37]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

kansas_city_merged = kansas_city_data

# merge kansas_city_grouped with kansas_city_data to add latitude/longitude for each neighborhood
kansas_city_merged = kansas_city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

kansas_city_merged.head() # check the last columns!

NameError: name 'kansas_city_data' is not defined

__Visualize the clusters__

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kansas_city_merged['Latitude'], kansas_city_merged['Longitude'], kansas_city_merged['Neighborhood'], kansas_city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'cm' is not defined

__Examine the clusters__

In [41]:
kansas_city_merged.loc[kansas_city_merged['Cluster Labels'] == 0, kansas_city_merged.columns[[1] + list(range(5, kansas_city_merged.shape[1]))]]

NameError: name 'toronto_merged' is not defined

In [ ]:
kansas_city_merged.loc[kansas_city_merged['Cluster Labels'] == 1, kansas_city_merged.columns[[1] + list(range(5, kansas_city_merged.shape[1]))]]